# GenAI with Azure Databricks - Developing RAG System

### Loading the csv file into the DBFS (Databricks File System)

In [ ]:
 %sh
 rm -r /dbfs/rag_lab
 mkdir /dbfs/rag_lab
 wget -O /dbfs/rag_lab/diabetes_faq.csv https://raw.githubusercontent.com/kuljotSB/DatabricksUdemyCourse/refs/heads/main/GenAI/diabetes_treatment_faq.csv

### Loading the csv file into a dataframe

In [ ]:
from pyspark.sql.functions import *

df = spark.read.load('/rag_lab/diabetes_faq.csv', format='csv', header=True)
display(df.limit(10))
df.printSchema()

### Installing the openai SDK in our python kernel

In [ ]:
%pip install openai==1.56.0

### Restarting our python kernel

In [ ]:
dbutils.library.restartPython()

### Creating an Azure OpenAI Client


In [ ]:
from openai import AzureOpenAI
import json

openai_endpoint = "<YOUR-AZURE-OPENAI-ENDPOINT>"
openai_key = "<YOUR-AZURE-OPENAI-KEY>"

client = AzureOpenAI(
    api_key = openai_key,
    api_version = "2024-02-15-preview",
    azure_endpoint = openai_endpoint
)

### Saving the updated/new dataframe into ADLS as parquet storage

In [ ]:
# Save the updated DataFrame as a Parquet file or table
df.write.mode("overwrite").parquet("/rag_lab/diabetes_faq.parquet")
df_with_embeddings.write.format("delta").mode("overwrite").saveAsTable("<catalog_name>.default.diabetes_faq_table")


### Installing the databricks vectorsearch SDK

In [ ]:
%pip install databricks-vectorsearch

### Restarting our python environment

In [ ]:
dbutils.library.restartPython()

### Developing the Cluster managed Vector index

In [ ]:
from databricks.vector_search.client import VectorSearchClient

vector_client = VectorSearchClient()

vector_client.create_endpoint(
     name="vector_search_endpoint",
     endpoint_type="STANDARD"
 )

index = vector_client.create_delta_sync_index(
   endpoint_name="vector_search_endpoint",
   source_table_name="<catalog_name>.default.diabetes_faq_table",
   index_name="<catalog_name>.default.diabetes_faq_index",
   pipeline_type="TRIGGERED",
   primary_key="Topic",
   embedding_source_column="Description",
   embedding_model_endpoint_name="databricks-gte-large-en"
  )

### Triggering our Vector Index - Information Retriever

In [ ]:
user_question = "what is diabetes?"

results_dict = vector_index.similarity_search(
            query_text = "{user_question}",
            columns = ["Topic", "Description"],
            num_results=1
          )

content = str(results_dict['result']['data_array'][0])
print(content)

### Developing the Generation Component of our RAG architecture


In [ ]:
gpt_response = client.chat.completions.create(
                model="gpt-4", # model = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. You will be passed the user query and the supporting knowledge that can be used to answer the user_query"},
                    {"role": "user", "content": f"user query : {user_question} and supporting knowledge: {content}"}
                ]
            )
          print(gpt_response.choices[0].message.content)

### Developing the RAG model

In [ ]:
import mlflow
from mlflow import pyfunc
from openai import AzureOpenAI

class RAGModel(pyfunc.PythonModel):
      def __init__(self, vector_index):
          self.vector_index=vector_index
      
      def retrieve(self, query):
          results_dict = self.vector_index.similarity_search(
            query_text = query,
            columns = ["Topic", "Description"],
            num_results=1
          )

          return results_dict
        
      def chatCompletionsAPI(self, user_query, supporting_knowledge):
          openai_client = AzureOpenAI(
            azure_endpoint = "<YOUR-AZURE-OPENAI-ENDPOINT>",
            api_key = "<YOUR-AZURE-OPENAI-API-KEY>",
            api_version = "2024-02-15-preview"
          )

          response = openai_client.chat.completions.create(
                model="YOUR_MODEL_NAME", # model = "deployment_name".
                messages=[
                    {"role": "system", "content": "You are a helpful assistant. You will be passed the user query and the supporting knowledge that can be used to answer the user_query"},
                    {"role": "user", "content": f"user query : {user_query} and supporting knowledge: {supporting_knowledge}"}
                ]
            )
          return response.choices[0].message.content
      
      def predict(self, context, data):
          query = data["query"].iloc[0]
          retrieved_docs = self.retrieve(query)
          return self.chatCompletionsAPI(query, text_data)
          


      

### Saving our Model

In [ ]:
test_model = RAGModel(vector_index=vector_index)

In [ ]:
from mlflow.models import infer_signature
import pandas as pd

signature = infer_signature(pd.DataFrame([{"query": "what is diabetes?"}]))
model_path = "RAGKULJOTmodel"
mlflow.pyfunc.save_model(path=model_path, python_model=test_model, signature=signature)

### Loading Our Saved Model

In [ ]:
# Load our custom model from the local artifact store
loaded_pyfunc_model = mlflow.pyfunc.load_model(model_path)


### Testing our Loaded/Saved Model

In [ ]:
model_input = pd.DataFrame([{"query": "what is diabetes?"}])

model_response = loaded_pyfunc_model.predict(model_input)

print(model_response)

### Logging our saved model as an artifact

In [ ]:
import mlflow

# Log the model as an artifact
with mlflow.start_run() as run:
    mlflow.log_artifacts(local_dir=model_path, artifact_path="rag_model")
    print(f"Model logged with run ID: {run.info.run_id}")


### Inferencing the real-time endpoint

In [ ]:
{
  "dataframe_records":[
    {
        "query":"what is diabetes?"
    }
  ]
}